In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ml4nlp2/final_folder")
!ls

from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')


# installs
!pip install datasets
!pip install transformers==4.28.0
!pip install sentencepiece==0.1.98
!pip install pysentimiento

#imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import  torch. nn. functional  as  F

#set device
import torch
# device = "cpu"
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

## hard_version

In [2]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder
task_encoder = MultiLabelBinarizer()

   
def task_hard_encode(df):
    import numpy as np
    # Create a new DataFrame to store the transformed labels
    transformed_df = df.copy()
    task_encoder.fit(all_task_hard_labels)
    all_labels = []
    for i in range(len(df['hard_label'])):
        # Apply eval to each element of the label column
        all_labels.append(eval(df['hard_label'][i]))
    
    transformed_labels = task_encoder.transform(all_labels)
    # print(pd.DataFrame(transformed_labels))
    # transformed_df["hard_label"] = pd.DataFrame(transformed_labels)
    transformed_labels = pd.Series([x.tolist() for x in transformed_labels])
    transformed_df["hard_label"] = transformed_labels
    return transformed_df

def task_hard_decode(df):
    # Inverse transform the binary-encoded vectors back to original labels
    import numpy as np
    decoded_df = df.copy()
    decode_labels = np.array(decoded_df["hard_label"].tolist())

    # Inverse transform the label column using the provided task_encoder
    decoded_labels = task_encoder.inverse_transform(decode_labels)
    
    # Assign the decoded labels back to the DataFrame
    decoded_df["hard_labels"] = decoded_labels

    return decoded_df

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset



#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")
og_test1 = pd.read_csv("test_final.csv")

train1_df = og_train1[["tweet","hard_label"]].dropna().reset_index(drop=True)
dev1_df = og_dev1[["tweet","hard_label"]].dropna().reset_index(drop=True)

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]]).dropna()
all_task_hard_labels = all_task_hard_labels.apply(lambda x: eval(x)).tolist()
print(all_task_hard_labels[0])

train1_df = task_hard_encode(train1_df)
# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task_hard_encode(dev1_df)
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)



import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    # accelerator = Accelerator(),
    output_dir="./output",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate= 1.5592115324779907e-05, #replaced with optuna best, original is 2e-05
    weight_decay= 0.00019014604610473898, #replaced with opunta best, original is 0.01
    logging_dir="./logs",
    # metric_for_best_model="f1_score",
    # greater_is_better=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
from scipy.special import expit
val_predictions = trainer.predict(val_dataset)
val_pred_probabilities = expit(val_predictions.predictions)

val_pred_labels = np.where(val_pred_probabilities > 0.5, 1, 0)
val_true_labels = val_dataset["label"]

# Calculate evaluation metrics 
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

['OBJECTIFICATION', 'SEXUAL-VIOLENCE']
train1 (6050, 2)
<bound method NDFrame.head of                                                   tweet          hard_label
0     @USER Ignora al otro, es un capullo.El problem...  [0, 0, 0, 1, 1, 0]
1     @USER Si comicsgate se parece en algo a gamerg...  [0, 0, 1, 0, 0, 0]
2     @USER Lee sobre Gamergate, y como eso ha cambi...  [0, 0, 1, 0, 0, 0]
3     @USER @USER @USER Entonces como así es el merc...  [1, 0, 0, 1, 0, 1]
4     @USER Aaah sí. Andrew Dobson. El que se dedicó...  [0, 0, 1, 0, 0, 0]
...                                                 ...                 ...
6045  idk why y’all bitches think having half your a...  [0, 1, 0, 1, 1, 1]
6046  This has been a part of an experiment with @US...  [0, 0, 0, 1, 0, 0]
6047  "Take me already" "Not yet. You gotta be ready...  [0, 0, 0, 0, 1, 0]
6048    @USER why do you look like a whore? /lh HTTPURL  [0, 1, 0, 1, 1, 1]
6049  ik when mandy says “you look like a whore” i l...  [0, 0, 0, 1, 0, 1]

[

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_

Epoch,Training Loss,Validation Loss
1,0.362400,0.349958
2,0.263400,0.325250
3,0.194300,0.361819
4,0.147400,0.386786


Validation Accuracy: 0.5858778625954199
Validation F1 Score: 0.6937763674656586


In [3]:
# Get predictions on the test set
from scipy.special import expit
test_predictions = trainer.predict(test_dataset)
test_pred_probabilities = expit(test_predictions.predictions) #same as sigmoid
test_pred_labels = np.where(test_pred_probabilities > 0.5, 1, 0)

In [ ]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

test_probs_list = create_results(test_pred_probabilities.tolist(), task_encoder.classes_)
test_probs_list

In [5]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_labels"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = list(decoded_labels[i])
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "/content/drive/MyDrive/ml4nlp2/for_submission/group-submissions/exist2023_roh-neil/task3_roh-neil_1.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(pd.Series(test_pred_labels.tolist()),columns=["hard_label"]), test_probs_list, og_test1[["id_EXIST"]],og_test1)

decoded labels length: 2076
output generated as json


In [ ]:
outputs_json

## soft_version

In [6]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    OBJECTIFICATION,'SEXUAL-VIOLENCE',NO,STEREOTYPING-DOMINANCE,IDEOLOGICAL-INEQUALITY,MISOGYNY-NON-SEXUAL-VIOLENCE
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["OBJECTIFICATION"], logits_dict["SEXUAL-VIOLENCE"],logits_dict["NO"],logits_dict["STEREOTYPING-DOMINANCE"],logits_dict["IDEOLOGICAL-INEQUALITY"], logits_dict["MISOGYNY-NON-SEXUAL-VIOLENCE"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"OBJECTIFICATION": logits_list[0], "SEXUAL-VIOLENCE": logits_list[1],"NO": logits_list[2], "STEREOTYPING-DOMINANCE": logits_list[3],"IDEOLOGICAL-INEQUALITY": logits_list[4], "MISOGYNY-NON-SEXUAL-VIOLENCE": logits_list[5]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)


from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task3.csv")
og_dev1 = pd.read_csv("dev_data_task3.csv")
og_test1 = pd.read_csv("test_final.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
dev1_df = og_dev1[["tweet","soft_label"]].dropna()


test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])


# print("train1",train1_df.shape)
print(train1_df.head)
# print("test1",test1_df.shape)
print(test1_df.head)



import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    # accelerator = Accelerator(),
    output_dir="./output",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate= 8.036768439133452e-06, #replaced with optuna best, original is 2e-05,1.5592115324779907e-05
    weight_decay= 8.398333136282262e-05, #replaced with opunta best, original is 0.01, 0.00019014604610473898
    logging_dir="./logs",
    # metric_for_best_model="f1_score",
    # greater_is_better=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)


import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


#with cross entropy loss and label smoothing
def custom_loss_fn(logits, soft_labels):
    probs = torch.sigmoid(logits)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss

# Define the trainer for each fold
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
from scipy.special import expit

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_probabilities = expit(val_predictions.predictions)

val_pred_labels = np.where(val_pred_probabilities > 0.5, 1, 0)
val_true_labels = [[1 if value > 0.5 else 0 for value in sublist] for sublist in val_dataset["label"]]

# Calculate evaluation metrics 
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.33333333333333304, 0.33333333333333304, 0.1...  
1     [0.16666666666666602, 0.0, 0.833333333333333, ...  
2                        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
3     [0.16666666666666602, 0

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_

Epoch,Training Loss,Validation Loss
1,14.416700,27.731012
2,13.972200,27.554157
3,13.791500,27.496634
4,13.684900,27.506582


Validation Accuracy: 0.4304857621440536
Validation F1 Score: 0.6487939419329587


In [9]:
# Get predictions on the test set
test_predictions1 = trainer.predict(test_dataset)

test_pred_probabilities1 = expit(test_predictions1.predictions) #, axis=1 #same as sigmoid
test_pred_labels1 = np.where(test_pred_probabilities1 > 0.5, 1, 0)

In [10]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

classes_labels = ["OBJECTIFICATION","SEXUAL-VIOLENCE","NO","STEREOTYPING-DOMINANCE","IDEOLOGICAL-INEQUALITY","MISOGYNY-NON-SEXUAL-VIOLENCE"]
test_probs_list1 = create_results(test_pred_probabilities1.tolist(), classes_labels)
test_probs_list1

[{'OBJECTIFICATION': 0.004808114375919104,
  'SEXUAL-VIOLENCE': 0.004077491350471973,
  'NO': 0.9993472695350647,
  'STEREOTYPING-DOMINANCE': 0.0041978247463703156,
  'IDEOLOGICAL-INEQUALITY': 0.004449627827852964,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.00396343320608139},
 {'OBJECTIFICATION': 0.004896828904747963,
  'SEXUAL-VIOLENCE': 0.0032235076650977135,
  'NO': 0.999313235282898,
  'STEREOTYPING-DOMINANCE': 0.005054525565356016,
  'IDEOLOGICAL-INEQUALITY': 0.004242564085870981,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.004666805732995272},
 {'OBJECTIFICATION': 0.006376348901540041,
  'SEXUAL-VIOLENCE': 0.0032852976582944393,
  'NO': 0.9993858337402344,
  'STEREOTYPING-DOMINANCE': 0.0055877044796943665,
  'IDEOLOGICAL-INEQUALITY': 0.004814504645764828,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.0034044580534100533},
 {'OBJECTIFICATION': 0.005621368996798992,
  'SEXUAL-VIOLENCE': 0.024935407564044,
  'NO': 0.39289534091949463,
  'STEREOTYPING-DOMINANCE': 0.06373234838247299,
  'IDEOLOGICAL-INEQUALI

In [11]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_labels"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = list(decoded_labels[i])
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "/content/drive/MyDrive/ml4nlp2/for_submission/group-submissions/exist2023_roh-neil/task3_roh-neil_2.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(pd.Series(test_pred_labels.tolist()),columns=["hard_label"]), test_probs_list, og_test1[["id_EXIST"]],og_test1)

decoded labels length: 2076
output generated as json


In [12]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_labels"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = list(decoded_labels[i])
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "/content/drive/MyDrive/ml4nlp2/for_submission/group-submissions/exist2023_roh-neil/task3_roh-neil_3.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(pd.Series(test_pred_labels1.tolist()),columns=["hard_label"]), test_probs_list, og_test1[["id_EXIST"]],og_test1)

decoded labels length: 2076
output generated as json
